# Projeto de *Processamento de Linguagem Natural*

## Introdução & Proposta

A seguinte proposta de solução tem por finalidade resolver o desafio levantado a partir do seguinte enunciado de projeto-problema proposto:

> Utilizando o dataset de revisões de filmes em português (baixado com ajuda do código abaixo), criar um classificador de sentimentos que consiga um score na métrica **F1 Score** superior a 70%.
>
> Fique a vontade para testar métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
>
> O trabalho poderá ser feito em grupo (mesmos grupos do Startup One).
>
> A data de entrega é dia 15/02/2019 até as 23:59 horas.

A base de dados disponibilizada é constituída por comentários do IMDb em língua portuguesa. O modelo baseado em Naives Bayes foi escolhido para a realização de predição de sentimento (análise de sentimento), o qual permitiu a obtenção de resultados significativos (F1 Score superior a 70%).

O modelo Naive Bayes tem sido utilizado para esta finalidade por atribuir pesos às palavras presentes no texto e conseguir correlacionar o peso destas para a classe de dados do problema – neste caso, dividida em `"neg"` e `"pos"`; respectivamente, negativo e positivo. De início testamos o modelo Naive Bayes sem a remoção de léxicos dos textos (como *stopwords*); em seguida, testamos o modelo com a remoção destes léxicos. A partir dos resultados obtidos, podemos constatar que não houve ganhos significativos a partir destas duas abordagens de preparação da base de dados.

## Autores da solução proposta

| Nome dos Integrantes     | RM            | Turma |
| :----------------------- | :------------ | :---: |
| Alexandre Lima Freitas   | RM 330376     | `1IA` |
| Ewerton Carlos Assis     | RM 330737     | `1IA` |
| Felipe Ribeiro da Silva  | RM 330332     | `1IA` |

## Obtenção da base de dados

A classe a seguir nos ajudará a acompanhar o progresso do download.

In [1]:
import tqdm
from tqdm import tqdm

class DLProgress(tqdm):
  last_block = 0

  def hook(self, block_num=1, block_size=1, total_size=None):
    self.total = total_size
    self.update((block_num - self.last_block) * block_size)
    self.last_block = block_num

O código a seguir fará o download do arquivo compactado do dataset e descompactará o mesmo. Em seguida, é feita a assertiva da obtenção da base dados em arquivo `csv`.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile
import zipfile

path = "content/imdb-pt-br.zip"
url = "https://drive.google.com/uc?export=download&id=1Jz3H23oBy3yDf8ARQsflwRX-EFX7HKqM"

if not isfile(path):
  with DLProgress(unit="B", unit_scale=True, miniters=1, desc="IMDB") as pbar:
    urlretrieve(
      url,
      path,
      pbar.hook)

if not isfile("content/reviews/imdb-review-pt-br.csv"):     
  zip_ref = zipfile.ZipFile(path, "r")
  zip_ref.extractall("content/reviews")
  zip_ref.close()

IMDB: 50.0MB [00:14, 3.41MB/s]


In [3]:
!ls -all -h content/reviews/imdb-reviews-pt-br.csv

-rw-r--r-- 1 earaujoassis staff 122M Aug  4 17:23 content/reviews/imdb-reviews-pt-br.csv


## Definições comuns aos modelos de análise

In [4]:
import numpy as np
import pandas as pd
import nltk
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from wordcloud import WordCloud

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df_master = pd.read_csv("content/reviews/imdb-reviews-pt-br.csv", encoding="utf-8", index_col=0)

In [6]:
list(df_master.columns.values)

['text_en', 'text_pt', 'sentiment']

In [7]:
df_master[["text_pt", "sentiment"]]

,text_pt,sentiment
id,,
1,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
2,Este é um exemplo do motivo pelo qual a maiori...,neg
3,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
4,Nem mesmo os Beatles puderam escrever músicas ...,neg
5,Filmes de fotos de latão não é uma palavra apr...,neg
6,Uma coisa engraçada aconteceu comigo enquanto ...,neg
7,Este filme de terror alemão tem que ser um dos...,neg
8,"Sendo um fã de longa data do cinema japonês, e...",neg
9,"""Tokyo Eyes"" fala de uma menina japonesa de 17...",neg


In [8]:
number_rows = df_master.shape[0]
traninng_threshold = int(0.40 * number_rows)

In [9]:
training_df = df_master[["text_pt", "sentiment"]].iloc[0:traninng_threshold]
testing_df = df_master[["text_pt", "sentiment"]].iloc[traninng_threshold + 1:-1]

In [10]:
training_df

,text_pt,sentiment
id,,
1,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
2,Este é um exemplo do motivo pelo qual a maiori...,neg
3,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
4,Nem mesmo os Beatles puderam escrever músicas ...,neg
5,Filmes de fotos de latão não é uma palavra apr...,neg
6,Uma coisa engraçada aconteceu comigo enquanto ...,neg
7,Este filme de terror alemão tem que ser um dos...,neg
8,"Sendo um fã de longa data do cinema japonês, e...",neg
9,"""Tokyo Eyes"" fala de uma menina japonesa de 17...",neg


In [11]:
testing_df

,text_pt,sentiment
id,,
19786,Um dos melhores filmes que eu já vi ... Bom ro...,pos
19787,"NA LINHA DE FOGO, na minha opinião, é um excel...",pos
19788,Este é outro dos muitos filmes de Eastwood que...,pos
19789,Um ótimo desempenho de Clint Eastwood e partic...,pos
19790,Frank Horrigan Clint Eastwood é um agente do s...,pos
19791,Clint Eastwood é um grande sucesso neste thril...,pos
19792,"Muito simplesmente um filme bem escrito, bem e...",pos
19793,Na linha de fogo nos dá um grande jogo de gato...,pos
19794,"Eu pude ver ""Anywhere But Here"" ontem à noite ...",pos


## Naive Bayes: preparação, treinamento & análise dos dados para solução simples

Nesta solução, o modelo baseado em Naive Bayes é utilizado sem qualquer remoção de palavras ou tratamento específico para a base de dados. Ainda assim, obtemos um valor de F1 Score significativo.

In [12]:
training_docs = training_df["text_pt"].values
training_classes = training_df["sentiment"].values
testing_docs = testing_df["text_pt"].values

In [13]:
vectorizer = CountVectorizer(analyzer="word")
frequency_training_docs = vectorizer.fit_transform(training_docs)
model = MultinomialNB()
model.fit(frequency_training_docs, training_classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
frequency_testing_docs = vectorizer.transform(testing_docs)
testing_classes = model.predict(frequency_testing_docs)

In [15]:
testing_classes

array(['pos', 'pos', 'pos', ..., 'pos', 'neg', 'pos'], dtype='<U3')

### Análise dos resultados a partir do modelo em Naive Bayes

In [16]:
y_true = testing_df["sentiment"].values
y_pred = testing_classes

In [17]:
f1_score(y_true, y_pred, average=None)

array([0.75941697, 0.75888128])

## Naive Bayes: preparação, treinamento & análise dos dados com remoção de *stopwords*

In [18]:
def remove_stopwords_and_punctuation(docs):
    stop_words = set(stopwords.words("portuguese"))
    neo_docs = []

    for sentence in docs:
        cleanr = re.compile('<.*?>')
        sentence = re.sub(r'\d+', '', sentence)
        sentence = re.sub(cleanr, '', sentence)
        sentence = re.sub("'", '', sentence)
        sentence = re.sub(r'\W+', ' ', sentence)
        sentence = sentence.replace('_', '')
        sentence = [word for word in sentence.lower().split() if word not in stop_words]
        sentence = ' '.join(sentence)
        neo_docs.append(sentence)

    return neo_docs

In [19]:
training_docs = remove_stopwords_and_punctuation(training_df["text_pt"].values)
training_classes = training_df["sentiment"].values
testing_docs = remove_stopwords_and_punctuation(testing_df["text_pt"].values)

In [20]:
vectorizer = CountVectorizer(analyzer="word")
frequency_training_docs = vectorizer.fit_transform(training_docs)
model = MultinomialNB()
model.fit(frequency_training_docs, training_classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
frequency_testing_docs = vectorizer.transform(testing_docs)
testing_classes = model.predict(frequency_testing_docs)

In [22]:
testing_classes

array(['pos', 'pos', 'pos', ..., 'pos', 'neg', 'pos'], dtype='<U3')

In [23]:
y_true = testing_df["sentiment"].values
y_pred = testing_classes

In [24]:
f1_score(y_true, y_pred, average=None)

array([0.76341975, 0.76545198])

## Comparativo das soluções

Ainda que os resultados apresentados pela segunda abordagem, com remoção das *stopwords*, tenha sido de maior qualidade no valor F1 Score, o ganho obtido não foi significativo: o valor de aumento em 1,3% pode ser afirmado como insignificante.

A solução proposta neste projeto baseia-se na comparação de métodos de preparação da base de dados para aplicação no modelo de solução. No caso do modelo baseado em Naive Bayes, a solução proposta consegue balancear e equilibrar a importância e o valor-peso das *stopwords* nas frases/sentenças analisadas. Assim, as *stopwords*, no caso desta solução baseada em Naive Bayes, não geram peso significante na qualidade da solução proposta.